In [77]:
#BIBLIOTEKI I OPCJE
import pandas as pd
import re
import random
import math
import uuid
import datetime
import numpy as np

import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

#TABELE
# playerTable = pd.DataFrame(columns=['Name', 'Surname', 'City', 'Category']) #to provide player names
# playerTable = pd.read_csv('examplePlayerTable.csv')
#df = pd.read_csv('csv/examplePlayerTable.csv')
#trickTable = pd.read_csv('trickTableJapan.csv')
#df = df.loc[df.index <= 10]
df = pd.read_csv('csv/masterTable.csv')
#MODUŁY
%run tableFunction.ipynb
%run gameFunction.ipynb
df

,name,points,word,gameUID,winGames,finGames,end,masterUID,timestamp
0,Hela,1,KEN,0,0,0,False,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26
1,Klaudia,3,KEN,0,0,0,True,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26
2,Hela,2,KEN,1,1,1,False,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26
3,Klaudia,3,KEN,1,0,1,True,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26
4,Hela,3,KEN,2,2,2,True,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26
5,Klaudia,2,KEN,2,0,2,False,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26
6,Piotr,1,KEN,0,0,0,False,58a81cd5-997d-4c33-941e-568c4b88d584,2022-01-31 18:27
7,Damian,3,KEN,0,0,0,True,58a81cd5-997d-4c33-941e-568c4b88d584,2022-01-31 18:27
8,Klaudia,3,KEN,0,0,0,True,58a81cd5-997d-4c33-941e-568c4b88d584,2022-01-31 18:27
9,Klaudia,2,KEN,0,0,0,False,20125f21-c12d-4f49-b310-fa2cb25229a8,2022-01-31 21:20


In [85]:
#filtr czasowy
def filterByDate(df, startDate=0, startTime='00:00', endDate=0, endTime='23:59'):
    if startDate == 0: startDate = df['timestamp'].values[0]
    if endDate == 0: endDate = df['timestamp'].values[-1]
    df['timestamp'] = pd.to_datetime(df['timestamp'], yearfirst=True)
    # startTime = '00:00'
    # startDate = '2022-02-01'
    # endTime = '23:59'
    # endDate = '2022-02-01'
    df = df.loc[(df['timestamp'] >= f'{startDate} {startTime}') & (df['timestamp'] <= f'{endDate} {endTime}')]
    return df

# osoba, która wygrała najwięcej gier
def overallWinner(df):
    df = df.loc[df['end'] == False]
    dfN = df[['name', 'end']].groupby('name').count().sort_values('end', ascending=False)
    dfDict = dfN.to_dict()
    return dfDict

#osoba, która wygrała najwięcej w danym czasie
def timeWinner(df, startDate=0, startTime='00:00', endDate=0, endTime='23:59'):
    print(startDate)
    dfFilter = filterByDate(df, startDate, startTime, endDate, endTime)
    dfDict = overallWinner(dfFilter)
    return dfDict
timeWinner(df)

0


{'end': {'Klaudia': 3, 'Hela': 2, 'Paweł': 2, 'Damian': 1, 'Piotr': 1}}

{'end': {'Klaudia': 3, 'Hela': 2, 'Paweł': 2, 'Damian': 1, 'Piotr': 1}}

In [13]:
df.loc[df['masterUID'] == '716597bf-bab6-453f-a136-d59a25b263e9']

,name,points,word,gameUID,winGames,finGames,end,masterUID,timestamp
22,Hela,1,KEN,0,0,0,False,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26:31.735016
23,Klaudia,3,KEN,0,0,0,True,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26:31.735016
24,Hela,2,KEN,1,1,1,False,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26:36.109558
25,Klaudia,3,KEN,1,0,1,True,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26:36.109558
26,Hela,3,KEN,2,2,2,True,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26:42.512791
27,Klaudia,2,KEN,2,0,2,False,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26:42.512791


In [11]:
df.value_counts('name')

name
Hela       10
Damian      8
Klaudia     8
Piotr       5
dtype: int64